In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as Autograd
from torch.utils.data import Dataset, DataLoader

# Generating the Data

In [13]:
'''
Data Set Class for single bidder 2 items
'''
class TypeData(Dataset):
    def __init__(self, num_samples=10000, num_agents=1, num_items=2):
        self.data = torch.rand(num_samples, num_agents * num_items)
    
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        return self.data[idx]

# Model Creation

## Additive Model & Valuation

### General Allocation Model

In [14]:
'''
Allocation Model Class Definition
'''
class AdditiveAllocationRegretNet(nn.Module):
    
    def __init__(self, num_agents, num_items, num_hidden_layers, num_neurons_per_hidden):
        super(AdditiveAllocationRegretNet, self).__init__()
        self.agents = num_agents # N agents,
        self.items = num_items # L items, 
        self.types = self.agents * self.items # NxL vector of all types arranged (N,L)
        self.hidlayers = num_hidden_layer # R = number of hidden layers
        self.hidsize = neurons_per_hidden # K = number of neurons per hidden layer
        
        # input layer (LxN types) -> 1st hidden
        self.layer1 = nn.Linear(in_features=self.types, out_features=self.hidsize)
        
        # adding R * Linear(K,K) hidden layers => adding R hidden layers with K neurons 
        self.layers = []
        for _ in range(self.hidlayers):
            temp_layer = nn.Linear(in_features=self.hidsize, out_features=self.num_hidden_size)
            self.layers.append(temp_layer)
        
        # last hidden -> output layer (same size as input layer = LxN types)
        self.final_layer = nn.Linear(in_features=self.hidsize, out_features=self.types)
        
    def forward(self, x):
        # activated output of first layer
        out = F.tanh(self.layer1(x))
        # traverse all hidden layers
        for i in range(self.hidlayers):
            temp_out = self.layers[i](out)
            out = F.tanh(temp_out)
        # activated Final layer output
        # applying softmax
        out = self.final_layer(out)
        out = out.view(self.agents, self.items)
        out = torch.transpose(F.softmax(torch.transpose(x, 0, 1)), 0, 1)
        out = out.view(self.agents * self.items)
        return out

### General Payments Model

In [15]:
'''
Payments Model Class Definition
'''
class AdditivePaymentsRegretNet(nn.Module):
    
    def _init__(self, num_agents, num_items, num_hidden_layers, num_neurons_per_hidden):
        super(AdditivePaymentsRegretNet, self).__init__()
        self.agents = num_agents # N agents,
        self.items = num_items # L items, 
        self.types = num_types # NxL vector of all types arranged (N,L)
        self.hidlayers = num_hidden_layer # R = number of hidden layers
#         self.hidsize = neurons_per_hidden # K = number of neurons per hidden layer
        
        # input layer (NxL) -> 1st hidden layer (N)
        self.layer1 = nn.Linear(in_features=self.types, out_features=self.agents)
        
        # adding R * Linear(N, N)
        self.layers = []
        for _ in range(self.hidlayers):
            temp_layer = nn.Linear(in_features=self.agents, out_features=self.agents)
            self.layers.append(temp_layer)
        
        # last hidden layer (N) -> output layer (N)
        self.layerfinal = nn.Linear(in_features=self.agents, out_features=self.agents)
    
    def forward(self, bids, allocs):
        # Processing through neural network
        # activated output of first layer
        out = F.tanh(self.layer1(bids))
        # traverse all hidden layers
        for i in range(self.hidlayers):
            temp_out = self.layers[i](out)
            out = F.tanh(temp_out)
        # activated Final layer output
        out_payment = F.sigmoid(self.layerfinal(out))
        
        # Generating final payments via allocation probs
        bid_vals = allocs * bids
        allocation_based_payments = torch.sum(bid_vals.view(self.agents, self.items), dim=1)
        final_payments = out_payment * allocation_based_payments
        return final_payments

# Training Functions

## Utility Function

In [17]:
# temp version for additive utilities only
class UtilityFunction(Autograd.Function):
    @staticmethod
    def forward(ctx, allocation_model, payments_model, bids, types):
        # Calculating Utilities
        # generating allocation probabilities
        allocs = allocation_model(bids)
        # calculating adjusted valuation per prob of trade
        vals_all = allocs * types
        # generating agg valuation per agent
        vals_reshaped = vals_all.view(allocation_model.agents, allocation_model.items)
        vals = torch.sum(vals_reshaped, dim=1)
        # determining payments
        payments = payments_model(bids, allocs)
        # caluclating utilities = valuations - payments
        utilities = vals - payments
        
        # Saving Key Variables & Gradients
        # acquiring reshaped version of types for easier backward functionality
        reform_types = types.view(allocation_model.agents, allocation_model.items)
        # acquiring dpdg for dudg (will save types directly)
        d_pays_d_alloc = Autograd.grad(payments, allocs)
        # acquiring d allocation d bid for d utility d bid
        d_alloc_d_bid = Autograd.grad(allocs, bids).view(allocation_model.agents, allocation_model.items)
        # acquiring d payment d bid for d utility d bid
        d_pay_d_bid = Autograd.grad(payments, bids)
        # saving all gradients and types
        ctx.save_for_backward(reform_types, d_pay_d_alloc, d_alloc_d_bid, d_pay_d_bid)
        
        # returning for forward()
        return utilities
    
    @staticmethod
    def backward(ctx, grad_output):
        '''
        NOTE: do we need notion of upstream gradient fosho?
        '''
        types, dpaydalloc, dallocdbid, dpaydbid = ctx.saved_tensors
        dudg = grad_output * (torch.sum(types, dim=1) - dpaydalloc)
        dudp = -grad_output
        dudb = grad_output * (torch.sum(dallocdbid * types, dim=1) - dpaydbid)
        return dudg, dudp, dudb, None

## Regret Function

In [22]:
class RegretFunction(Autograd.Function):
    @staticmethod
    def forward(ctx, util, alloc_model, pay_model, bids, types):
        # Calculating Regret
        # strategic type utility calculation
        util_bids = util(alloc_model, pay_model, bids, types)
        # true type utility calculation
        util_types = util(alloc_model, pay_model, types, types)
        # regret = strategic util - true type util
        regret_total = (util_bids - util_types)/alloc_model.items
        
        # Saving Gradients
        # NOT SURE IF THIS WORKS MIGHT NEED TO REFORMAT
        # saving grad wrt strategic utility
        d_reg_d_bid = Audotgrad.grad(regret_total, util_bids)
        # saving grad wrt true type util
        d_reg_d_tru = Autograd.grad(regret_total, util_types)
        # saving grad wrt g(*)
        d_reg_d_alloc = Autograd.grad(regret_total, alloc_model(bids))
        # saving grad wrt p(*)
        d_reg_d_paym = Autograd.grad(regret_total, pay_model(bids))
        # saving values
        ctx.save_for_backward(d_reg_d_bid, d_reg_d_tru, d_reg_d_alloc, d_reg_d_paym, alloc_model.items)
        
        return regret_total
    
    @staticmethod
    def backward(ctx, grad_output):
        dregdbid, dregdtru, dregdallc, dregdpay, items = ctx.saved_tensors
        drdb = grad_output * (dregdbid/items)
        drdt = grad_output * (-dregdtru/items)
        drdg = grad_output * (dregdallc)
        drdp = grad_output * (dregdpay)
        return None, drdg, drdp, drdb, drdt

# Setting Up Training

## Defining Hyper-Parameters

In [19]:
# outer and inner epochs
epochs = 80
misreport_epochs = 25

# learning rates
val_lr = 0.1
pay_lr = 0.001
all_lr = 0.001

# batch sizes
batch_size = 100
# penalty terms
rho = 1

## Initializing DataLoaders

In [20]:
# initializing our data sets from earlier
trainset = TypeData(num_agents=1, num_items=2)
valset = TypeData(num_agents=1, num_items=2)

# Data Loaders are used to produce batches for training
traindl = DataLoader(trainset, batch_size=batch_size, shuffle=True)
valdl = DataLoader(valset, batch_size=batch_size, shuffle=True)

## Defining Loss Function

In [ ]:
'''
Unsure how to execute RegretLoss without passing in payments model and generating re-inference
'''
class RegretLoss(Autograd.Function):
    @staticmethod
    def forward(ctx, payments, pay_grads, lambdas, regrets, regrets_grads, items, rho):
        term1 = -torch.sum(payments)/items
        term2 = torch.dot(lambdas, regrets)
        term3 = (rho/2) * torch.dot(regrets, regrets)
        loss = term1 + term2 + term3
        ctx.save_for_backward(pay_grads, regrets_grads, )